<a href="https://colab.research.google.com/github/DonggiChae/practice/blob/master/mask_r_cnn_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchvision
import torch
import numpy as np
import torch.nn as nn
from torchvision.ops.boxes import nms, box_iou, clip_boxes_to_image, remove_small_boxes
import torch.nn.functional as F
import torchvision.transforms as T
import torch.nn.init as nt
from torchvision.ops import roi_align

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# trf = T.compose([T.resize(),T.ToTensor()])
# input_img = trf(img).unsqueeze(0)

image = torch.zeros((1, 3, 800, 800)).float() 

bbox = torch.FloatTensor([[20, 30, 400, 500], [300, 400, 500, 600]]) # dummy box  x1, y1, x2, y2
labels = torch.LongTensor([6, 8]) 

fe_size = 50


dummy_img = torch.zeros((1, 3, 800,800)).float()
# print(dummy_img)
#Out: torch.Size([1, 3, 800, 800])


model = torchvision.models.vgg16(pretrained=True)
req_features = model.features[:30]
faster_rcnn_fe_extractor = nn.Sequential(*req_features)
out_map = faster_rcnn_fe_extractor(image)

print(req_features)





In [ ]:
# #Anchor Generator
anchor_size = [128, 256, 512]
anchor_ratio = [0.5, 1, 2]
anchors = len(anchor_size) * len(anchor_ratio) * out_map.size(2) * out_map.size(3)
anchor_boxes = torch.zeros((len(anchor_ratio) * len(anchor_size) * 50 * 50, 4) )

ctr_x = np.arange(16, (fe_size+1) * 16, 16)
ctr_y = np.arange(16, (fe_size+1) * 16, 16)

ctr = np.zeros((2500, 2))

def ctr_point(ctr_x, ctr_y):
    index = 0
    for x in range(len(ctr_x)):
        for  y in range(len(ctr_y)):
            ctr[index, 0] = ctr_x[x] - 8
            ctr[index, 1] = ctr_y[y] - 8
            index += 1
    return ctr


ctr_point(ctr_x, ctr_y)
# print(ctr.shape)
index = 0 
for c_p in ctr:
    ctr_x, ctr_y = c_p
    for idx_s in range(len(anchor_size)):
        for idx_r in range(len(anchor_ratio)):
            h = anchor_size[idx_s] * np.sqrt(anchor_ratio[idx_r])
            w = anchor_size[idx_s] * np.sqrt(1 / anchor_ratio[idx_r])
            anchor_boxes[index, 0] =  ctr_x - w / 2
            anchor_boxes[index, 1] =  ctr_y - h / 2
            anchor_boxes[index, 2] =  ctr_x + w / 2
            anchor_boxes[index, 3] =  ctr_y + h / 2
            index += 1

# print(anchor_boxes)


valid_boxes = np.where((anchor_boxes[:, 0] >= 0) & 
                        (anchor_boxes[:, 1] >= 0) & 
                        (anchor_boxes[:, 2] <= 800) & 
                        (anchor_boxes[:, 3] <= 800))[0]

vaild_anchors = anchor_boxes[valid_boxes]

# print(vaild_anchors)
# print(bbox)





In [ ]:
#iou
pos_iou_threshold = 0.7
neg_iou_threshold = 0.3


ious = box_iou(vaild_anchors, bbox) * 10 #make something wrong result
# print(ious)


valid_labels = np.empty((valid_boxes.shape[0],))
valid_labels.fill(-1)
anchor_max_ious = ious.amax(axis=1)
valid_labels[np.where(anchor_max_ious >= pos_iou_threshold)[0]] = 1
valid_labels[np.where(anchor_max_ious < neg_iou_threshold)[0]] = 0



In [ ]:
#Determine vaild anchor, positve = 1, negative = 0 , (0.7 > i don't need (-1) > 0.3)
pos_ratio = 0.5
randomly_sample = 256


total_pos = len(np.where(valid_labels == 1)[0])
n_pos_sample = randomly_sample*pos_ratio if total_pos > randomly_sample*pos_ratio else total_pos
n_neg_sample = randomly_sample - n_pos_sample
n_pos_sample = int(n_pos_sample)
n_neg_sample = int(n_neg_sample)

pos_index = np.where(valid_labels == 1)[0]
if len(pos_index) > randomly_sample*pos_ratio:
    disable_index = np.random.choice(pos_index, size=len(pos_index)-n_pos_sample, replace=False)
    valid_labels[disable_index] = -1

neg_index = np.where(valid_labels == 0)[0]
disable_index = np.random.choice(neg_index, size=len(neg_index) - n_neg_sample, replace=False)
valid_labels[disable_index] = -1



In [ ]:
#bbox regression 
# t_{x} = (x - x_{a})/w_{a}
# t_{y} = (y - y_{a})/h_{a}
# t_{w} = log(w/ w_a)
# t_{h} = log(h/ h_a)


argmax_ious = ious.argmax(axis=1)
max_iou_bbox = bbox[argmax_ious]
# print(max_iou_bbox.shape)

height = vaild_anchors[:, 3] - vaild_anchors[:, 1]
width = vaild_anchors[:, 2] - vaild_anchors[:, 0]
ctr_y = vaild_anchors[:, 1] + 0.5 * height
ctr_x = vaild_anchors[:, 0] + 0.5 * width

base_height = max_iou_bbox[:, 3] - max_iou_bbox[:, 1]
base_width = max_iou_bbox[:, 2] - max_iou_bbox[:, 0]
base_ctr_y = max_iou_bbox[:, 1] + 0.5 * base_height
base_ctr_x = max_iou_bbox[:, 0] + 0.5 * base_width


# valid anchor boxes location
eps = np.finfo(np.float64).eps
height = np.maximum(height, eps) 
width = np.maximum(width, eps)
dy = (base_ctr_y - ctr_y) / height
dx = (base_ctr_x - ctr_x) / width
dh = np.log(base_height / height)
dw = np.log(base_width / width)
anchor_locs = np.vstack((dx, dy, dw, dh)).transpose()



anchor_labels = np.empty((len(anchor_boxes),), dtype=np.int32)
anchor_locations = np.zeros((len(anchor_boxes), 4), dtype=np.float32)

anchor_labels.fill(-1)
anchor_labels[valid_boxes] = valid_labels
print(anchor_labels.shape)

anchor_locations[valid_boxes] = anchor_locs

print(anchor_locations.shape)


In [ ]:
#rpn
#num_anchors = predicted box
layer_weight = 0.1
layer_bias = 0

in_channels = 512
num_anchors = len(anchor_size) * len(anchor_ratio)

conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1)
box_layer = nn.Conv2d(in_channels, num_anchors * 4, kernel_size=1, stride=1)
score_layer = nn.Conv2d(in_channels, num_anchors * 2, kernel_size=1, stride=1)
conv.weight.data.normal_(layer_weight, std=0.01)
box_layer.weight.data.normal_(layer_weight, std=0.01)
score_layer.weight.data.normal_(layer_weight, std=0.01)
conv.bias.data.zero_()
box_layer.bias.data.zero_()
score_layer.bias.data.zero_()


x = conv(out_map)
pred_anchor_locs = box_layer(x)
pred_cls_scores = score_layer(x)
# print(pred_cls_scores)
# print(pred_anchor_locs)

pred_anchor_locs = pred_anchor_locs.permute(0, 2, 3, 1).contiguous().view(1, -1, 4)
pred_cls_scores = pred_cls_scores.permute(0, 2, 3, 1).contiguous().view(1, -1, 2)
objectness_score = pred_cls_scores.view(1, 50, 50, 9, 2)[:, :, :, :, 1].contiguous().view(1, -1)



# print(pred_anchor_locs.shape)
# print(pred_cls_scores.shape)
# print(objectness_score.shape)


rpn_locs = pred_anchor_locs[0]
rpn_scores = pred_cls_scores[0]


# print(rpn_locs.shape)
# print(rpn_scores.shape)


In [ ]:
anchor_labels = torch.as_tensor(anchor_labels)
# print(anchor_labels)
rpn_cls_loss = F.cross_entropy(rpn_scores, anchor_labels.long(), ignore_index=-1)
# print(rpn_cls_loss)

sampled_pos_inds_subset = anchor_labels > 0
# print(sampled_pos_inds_subset)
# print(anchor_locations)
regression_targets = anchor_locations[sampled_pos_inds_subset]
pred_target = rpn_locs[sampled_pos_inds_subset]

# print(regression_targets)
# print(pred_target)
pred_target = pred_target
regression_targets = torch.from_numpy(regression_targets)



x = torch.abs(regression_targets - pred_target)
rpn_loc_loss = ((x<0.5).float()*(x**2)*0.5 + (x>0.5).float()*(x-0.5)).sum()

# print(rpn_loc_loss)

rpn_lambda = 10.
N_reg = sampled_pos_inds_subset.float().sum() # number of bounding box for positve anchors
rpn_loc_loss = rpn_loc_loss / N_reg
rpn_loss = rpn_cls_loss + (rpn_lambda * rpn_loc_loss) # something wrong???
# print(N_reg)
# print(rpn_loc_loss)
# print(rpn_loss)

In [ ]:
#Non-maximum supression(NMS)

nms_thresh = 0.7
n_train_pre_nms = 12000
n_train_post_nms = 2000
n_test_pre_nms = 6000
n_test_post_nms = 300
min_size = 16

anc_height = anchor_boxes[:, 3] - anchor_boxes[:, 1]
anc_width = anchor_boxes[:, 2] - anchor_boxes[:, 0]
anc_ctr_y = anchor_boxes[:, 1] + 0.5 * anc_height
anc_ctr_x = anchor_boxes[:, 0] + 0.5 * anc_width
print(anc_width)

# The 22500 anchor boxes location and labels predicted by RPN
pred_anchor_locs_numpy = pred_anchor_locs[0].data.numpy()
objectness_score_numpy = objectness_score[0].data.numpy()
dx = pred_anchor_locs_numpy[:, 0::4] 
dy = pred_anchor_locs_numpy[:, 1::4] 
dw = pred_anchor_locs_numpy[:, 2::4] 
dh = pred_anchor_locs_numpy[:, 3::4] 

# print(dx)
# print(dy)


roi = np.zeros(pred_anchor_locs_numpy.shape)
roi[:, 0::4] = dx - 0.5 * dw
roi[:, 1::4] = dy - 0.5 * dh
roi[:, 2::4] = dx + 0.5 * dw
roi[:, 3::4] = dy + 0.5 * dh

roi = torch.from_numpy(roi)
print(roi)

boxes = clip_boxes_to_image(roi, [800,800])
boxes = remove_small_boxes(boxes, 16)
score = objectness_score_numpy[boxes]
order = score.ravel().argsort()[::-1]
print(order)


In [ ]:
#Take nms topN 
order = order[:n_train_pre_nms]
order = np.sort(order)
order = torch.from_numpy(order)
# print(order)
# print(roi.shape)
rois = roi[order]
print(score.shape)
scores = torch.from_numpy(score[order]).type(torch.DoubleTensor)
print(rois.shape)
print(scores)
print(scores.shape)
keep = nms(rois, scores, nms_thresh)
print(keep.shape)
post_train_valid_rois = rois[keep][:n_train_post_nms] #??????????????
post_train_valid_scores = scores[keep][:n_train_post_nms] #????????
print(post_train_valid_rois.shape) 
print(post_train_valid_scores.shape) 

In [ ]:
# Proposal targets

n_sample = 128
pos_ratio = 0.25
pos_iou_thresh = 0.5
neg_iou_thresh_hi = 0.5
neg_iou_thresh_lo = 0.0


ious = box_iou(post_train_valid_rois, bbox)
print(ious.shape) 


bbox_assignments = ious.argmax(axis=1)
roi_max_ious = ious.max(axis=1)
roi_target_labels = labels[bbox_assignments]
print(roi_target_labels.shape)
print(roi_max_ious)


total_n_pos = len(np.where(roi_max_ious[0] >= pos_iou_thresh)[0])
n_pos_sample = n_sample*pos_ratio if total_n_pos > n_sample*pos_ratio else total_n_pos
n_neg_sample = n_sample - n_pos_sample

print(n_pos_sample) 
print(n_neg_sample) 

pos_index = np.where(roi_max_ious[0] >= pos_iou_thresh)[0]
pos_index = np.random.choice(pos_index, size=n_pos_sample, replace=False)

neg_index = np.where((roi_max_ious[0] < neg_iou_thresh_hi) & (roi_max_ious[0] > neg_iou_thresh_lo))[0]
neg_index = np.random.choice(neg_index, size=n_neg_sample)

keep_index = np.append(pos_index, neg_index)
post_sample_target_labels = roi_target_labels[keep_index].data.numpy()
post_sample_target_labels[len(pos_index):] = 0
post_sample_rois = post_train_valid_rois[keep_index]
print(post_sample_rois.shape)

post_sample_bbox = bbox[bbox_assignments[keep_index]]




height = post_sample_rois[:, 3] - post_sample_rois[:, 1]
width = post_sample_rois[:, 2] - post_sample_rois[:, 0]
ctr_y = post_sample_rois[:, 1] + 0.5 * height
ctr_x = post_sample_rois[:, 0] + 0.5 * width
# print(height)

base_height = post_sample_bbox[:, 3] - post_sample_bbox[:, 1]
base_width = post_sample_bbox[:, 2] - post_sample_bbox[:, 0]
base_ctr_y = post_sample_bbox[:, 1] + 0.5 * base_height
base_ctr_x = post_sample_bbox[:, 0] + 0.5 * base_width


eps = np.finfo(np.float64).eps
height = np.maximum(height, eps)
width = np.maximum(width, eps)


dy = (base_ctr_y - ctr_y) / height
dx = (base_ctr_x - ctr_x) / width
dh = np.log(base_height / height)
dw = np.log(base_width / width)


gt_roi_locs = np.vstack((dy, dx, dh, dw)).transpose()
# print(gt_roi_locs.size)




In [ ]:
#Fast R-CNN



roi_indexes = 0 * np.ones((len(post_sample_rois),))
roi_indexes = torch.from_numpy(roi_indexes).float()
print(rois.shape, roi_indexes.shape)
indexes_and_rois = torch.cat([roi_indexes[:, None], post_sample_rois], dim=1)
# indexes_and_rois = indexes_and_rois.DoubleTensor()
out_map = torch.tensor(out_map, dtype=torch.double)
# print(indexes_and_rois)
# print(out_map)
roi_aligns = roi_align(out_map, indexes_and_rois , output_size = 7, spatial_scale = 1, sampling_ratio = -1, aligned = False)




In [ ]:
print(roi_aligns.shape)
output_roi_aligns = roi_aligns.view(roi_aligns.size(0), -1)
print(output_roi_aligns.shape)
output_roi_aligns = torch.tensor(output_roi_aligns, dtype=torch.float)

roi_head = nn.Sequential(nn.Linear(25088, 4096),
                        nn.Linear(4096, 4096))

cls_loc = nn.Linear(4096, 21*4)
cls_loc.weight.data.normal_(0, 0.01)
cls_loc.bias.data.zero_()

cls_score = nn.Linear(4096, 21)
cls_score.weight.data.normal_(0, 0.01)
cls_score.bias.data.zero_()

x = roi_head(output_roi_aligns)
roi_cls_loc = cls_loc(x)
roi_cls_score = cls_score(x)

print(roi_cls_loc.shape, roi_cls_score.shape)





In [ ]:
print(gt_roi_locs.shape)
gt_roi_cls_loc = gt_roi_locs
gt_roi_cls_label = torch.from_numpy(post_sample_target_labels).long()
print(roi_cls_score.shape)

roi_cls_loss = F.cross_entropy(roi_cls_loc, gt_roi_cls_label, ignore_index=-1)

num_roi = roi_cls_loc.size(0)
roi_cls_loc = roi_cls_loc.view(-1, 21, 4)
roi_cls_loc = roi_cls_loc[torch.arange(num_roi), gt_roi_cls_label]
print(roi_cls_loc.shape)

mask = gt_roi_cls_label>0
mask_loc_pred = roi_cls_loc[mask]
mask_loc_target = gt_roi_cls_loc[mask]

print(mask_loc_pred.shape)#????????????
print(mask_loc_target.shape)#???????????????

x = torch.abs(mask_loc_pred-mask_loc_target)
roi_loc_loss = ((x<0.5).float()*x**2*0.5 + (x>0.5).float()*(x-0.5)).sum()
print(roi_loc_loss)

In [ ]:
mask_fcn = nn.Sequential(
 nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 nn.ReLU(inplace=True),
 nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 nn.ReLU(inplace=True),
 nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 nn.ReLU(inplace=True),
 nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 nn.ReLU(inplace=True),
 nn.ConvTranspose2d(256, 256, kernel_size=(2, 2), stride=(2, 2)),
 nn.ReLU(inplace=True),
 nn.Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
)

mask_head = nn.Sequential(*mask_fcn)
masks = mask_head(output_roi_aligns)


In [ ]:
#roi_align


roi = roi_align(input, boxes, output_size = 7, spatial_scale)




#Loss function

reg_loss = 0

def smooth_l1_loss(predicted_bbox, bbox_target):
    n= torch.abs(predicted_bbox - bbox_target)
    if n < 1:
        loss = 0.5 * n ** 2
    else:
        loss = n - 0.5
    return loss


classification_loss = F.cross_entropy(class_score, labels)


rpn_cls_loss = classification_loss
rpn_loc_loss = smooth_l1_loss(predicted_bbox, bbox_target).sum()

#rpn loss function
rpn_lambda = 10.
N_reg = (gt_rpn_score >0).float().sum() # number of bounding box for positve anchors
rpn_loc_loss = rpn_loc_loss / N_reg
rpn_loss = rpn_cls_loss + (rpn_lambda * rpn_loc_loss)

 
#  x = #pixel 
# mask_binary_loss = sigmoid()


def maskrcnn_inference(x, labels):
    # type: (Tensor, List[Tensor]) -> List[Tensor]
    """
    From the results of the CNN, post process the masks
    by taking the mask corresponding to the class with max
    probability (which are of fixed size and directly output
    by the CNN) and return the masks in the mask field of the BoxList.
    Args:
        x (Tensor): the mask logits
        labels (list[BoxList]): bounding boxes that are used as
            reference, one for ech image
    Returns:
        results (list[BoxList]): one BoxList for each image, containing
            the extra field mask
    """
    mask_prob = x.sigmoid()

    # select masks corresponding to the predicted classes
    num_masks = x.shape[0]
    boxes_per_image = [label.shape[0] for label in labels]
    labels = torch.cat(labels)
    index = torch.arange(num_masks, device=labels.device)
    mask_prob = mask_prob[index, labels][:, None]
    mask_prob = mask_prob.split(boxes_per_image, dim=0)

    return mask_prob



# (mask_head): MaskRCNNConvUpsampleHead(
#       (mask_fcn1): Conv2d(
#         256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
#         (activation): ReLU()
#       )
#       (mask_fcn2): Conv2d(
#         256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
#         (activation): ReLU()
#       )
#       (mask_fcn3): Conv2d(
#         256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
#         (activation): ReLU()
#       )
#       (mask_fcn4): Conv2d(
#         256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
#         (activation): ReLU()
#       )
#       (deconv): ConvTranspose2d(256, 256, kernel_size=(2, 2), stride=(2, 2))
#       (deconv_relu): ReLU()
#       (predictor): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))


# L = L_cls + L_box + L_mask
